In [0]:
!pip install git+https://github.com/tdrobbins/unet.git@nchannels
!nvidia-smi

In [0]:
%load_ext tensorboard
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import unet
import pandas as pd
import matplotlib.pyplot as pp
import seaborn as sns
from glob import glob
import numpy as np
import datetime

from google.colab import files

# Preprocessing

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

import sys
sys.path.insert(1, '/content/drive/My Drive/cs230/')
import utils

In [0]:
# Loading and preprocessing data

# Preprocessing paramters
CROP        = 512    # split raw images into many CROPxCROP images
RESIZE      = 128    # downsample cropped image to RESIZExRESIZE
N_CLASSES   = 5      # 2, 5, or 10
CELLTOWERS  = True  

STATE = "*"

# Loading satellite images into an array
sat_files = np.sort(glob("./drive/My Drive/cs230/sentinel2/{}/*.tif".format(STATE)))
cell_files = np.sort(glob("./drive/My Drive/cs230/celltowers/{}/*.jpg".format(STATE)))

X = utils.load_sat_imgs(sat_files, cell_files, crop_size=CROP, resize=RESIZE)  
m,nx,ny,nchannels = X.shape

# Loading nclasses version of lte mask images into an array
lte_files = np.sort(glob("./drive/My Drive/cs230/lte/{}/*.jpg".format(STATE,N_CLASSES)))
Y = utils.load_masks(lte_files, N_CLASSES, crop_size==CROP, resize=RESIZE, onehot=True)

# Model Training

In [0]:
# Setting up the datasets for tf
TRAIN_SPLIT   = 0.8
BATCH_SIZE    = 32
CLASS_WEIGHTS = "auto" # make sure len(CLASS_WEIGHTS) == N_CLASSES
SEED          = 1

# Experimenting with settings CLASS_WEIGHTS automatically
if CLASS_WEIGHTS == "auto":
    counts = Y.sum(axis=tuple(range(Y.ndim - 1)))
    CLASS_WEIGHTS = np.round(max(counts)/counts).astype(np.uint)

if CLASS_WEIGHTS is not None:
    Y = Y*CLASS_WEIGHTS
    
(X_train, Y_train), (X_dev, Y_dev) = utils.split_train_dev(X,Y,split=TRAIN_SPLIT, batch_size=BATCH_SIZE, seed=SEED)

# Create generators using split datasets
train_datagen = ImageDataGenerator()
dev_datagen = ImageDataGenerator()

train_set = train_datagen.flow(X_train,Y_train,batch_size=BATCH_SIZE)
dev_set = dev_datagen.flow(X_dev,Y_dev)

In [0]:
# Model parameters
LEARNING_RATE = 0.0001
LAYER_DEPTH   = 5
ROOT_FILTERS  = 64
DROPOUT       = 0.0

SAVE_ROOT = "/content/drive/My Drive/cs230/logs/"

SAVE_PATH = SAVE_ROOT+datetime.datetime.now().strftime("%y%m%d-%H%M")

# Building the model
unet_model = unet.build_model(nx, ny,
                          channels = nchannels,
                          num_classes = N_CLASSES,
                          layer_depth = LAYER_DEPTH,
                          filters_root= ROOT_FILTERS,
                          dropout_rate = DROPOUT,
                          padding = "same"
                          )

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=SAVE_PATH, histogram_freq=1)

unet_model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=LEARNING_RATE),
              loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=["Precision", "Recall", "CategoricalAccuracy"])

In [0]:
%tensorboard --logdir "$SAVE_ROOT"

In [0]:
# Training parameters 
EPOCHS_PER_LOOP = 100
LOOPS           = 15

for i in range(LOOPS):
  model_history = unet_model.fit(train_set,
                                validation_data=dev_set,
                                epochs=EPOCHS_PER_LOOP, 
                                batch_size=BATCH_SIZE,
                                steps_per_epoch=len(X_train)//BATCH_SIZE,
                                initial_epoch=i*EPOCHS_PER_LOOP,
                                callbacks=[tensorboard_callback])
  unet_model.save(SAVE_PATH+"model")